# CogSci 2021 figures & analysis

In [ ]:
# set up imports
import os
import sys
__file__ = os.getcwd()
proj_dir =  os.path.dirname(os.path.realpath(__file__))
sys.path.append(proj_dir)
utils_dir = os.path.join(proj_dir,'utils')
sys.path.append(utils_dir)
analysis_dir = os.path.join(proj_dir,'analysis')
analysis_utils_dir = os.path.join(analysis_dir,'utils')
sys.path.append(analysis_utils_dir)
agent_dir = os.path.join(proj_dir,'model')
sys.path.append(agent_dir)
agent_util_dir = os.path.join(agent_dir,'utils')
sys.path.append(agent_util_dir)
experiments_dir = os.path.join(proj_dir,'experiments')
sys.path.append(experiments_dir)
df_dir = os.path.join(proj_dir,'results/dataframes')

In [ ]:
from model.Subgoal_Planning_Agent import *
import utils.blockworld as bw
import utils.blockworld_library as bl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import model.utils.decomposition_functions

In [ ]:
import scipy.stats as st

In [ ]:
#inline plots
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Plot styling:

In [ ]:
plt.rcParams["figure.figsize"] = (20,7)
plt.rcParams.update({'font.size': 22})

In [ ]:
from matplotlib import rc
# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['font.sans-serif'] = ['Helvetica']
rc('text.latex', preamble=r'\usepackage{tgheros} \usepackage{newtxsf} \renewcommand{\familydefault}{\sfdefault} \usepackage{mathastext}') #sets the font via latex preamble—only way to autoset tick labels?

In [ ]:
#display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.min_rows', 6)

In [ ]:
#helper function for pd.agg
def item(x):
    return x.tail(1).item()

Let's load the results of the experiment

In [ ]:
df_paths = ['subgoal planning full BFS2.pkl',
'subgoal planning full bogo.pkl',
'subgoal planning full BFS1.pkl']

In [ ]:
df_paths = ['subgoal planning full BFS0 to 2 small.pkl']

In [ ]:
df_paths = ['simulated lookaheads tiny.pkl']

In [ ]:
#load all experiments as one dataframe
# df = pd.concat([pd.read_pickle(os.path.join(df_dir,l)) for l in df_paths])
print("Loaded dataframe")

In [ ]:
#alternatively, choose csv
df_paths = ['simulated lookaheads BFS2.csv']

In [ ]:
#load all experiments as one dataframe from CSV
df = pd.concat([pd.read_csv(os.path.join(df_dir,l)) for l in df_paths])
print("Loaded dataframe")

Manually add agent labels to dataframe

In [ ]:
#TBD

Let's choose just one parent agent to make it easier to interpret

In [ ]:
oneadf = df[(df['parent: lower level: horizon'] == 2) & (df['parent: lower level: scoring_function'] == 'F1_stability_score') & (df['include_subsequences'] == False)]

creating `fdf` with only outcomes

In [ ]:
# function for 95CI
def CI95(data):
    return st.t.interval(alpha=0.95,df=len(data)-1,loc=np.mean(data),scale=st.sem(data))

In [ ]:
fdf = oneadf.groupby('run_ID').agg({
    'agent_label' : item,
    'world' : item,
    'c_weight' : item,
    'sequence_length' : item,
    'include_subsequences' : item,
    'parent: lower level: agent_type' : item,
    'parent: lower level: scoring_function' : item,
    'parent: lower level: horizon' : item,
    'partial_planning_cost':['sum','mean',np.std],
    'partial_solution_cost':['sum','mean',np.std],
    'planning_cost':['sum','mean',np.std],
    'solution_cost':['sum','mean',np.std],
    'all_sequences_planning_cost':['sum','mean',np.std], #includes penalty and therefore is meaningless
    'world_status' : lambda x:x.tail(1).item(),
    'decomposed_silhouette' : 'count' #how many subgoals did we act out? With stepsize of 1 number of subgoals chosen
})

#flatten the dataframe to remove multi-index for next groupby
fdf.columns = [' '.join(col).strip() for col in fdf.columns.values]
fdf.reset_index(inplace=True)

In [ ]:
#how many wins do we have?
fdf['world_status <lambda>'].value_counts()

In [ ]:
# condition on winning solving the world
wfdf = fdf[fdf['world_status <lambda>'] == 'Win']

## Sequence length

In [ ]:
length_df = wfdf.groupby('sequence_length item').agg({
    'world_status <lambda>' : lambda x:len([r for r in x if r == 'Win'])/len(x),
     'partial_planning_cost sum':['mean',CI95],
     'partial_solution_cost sum':['mean',CI95],
     'partial_solution_cost mean':['mean',CI95],
     'planning_cost sum':['mean',CI95],
     'planning_cost mean':['mean',CI95],
#      'solution_cost':'mean', #includes penalty and therefore is meaningless
     'all_sequences_planning_cost sum':['mean',CI95],
     'all_sequences_planning_cost mean':['mean',CI95],
     'decomposed_silhouette count' : ['mean',CI95]
})

In [ ]:
length_df

In [ ]:
np.array([abs(Ys - CI95s[0]),abs(Ys - CI95s[1])])

In [ ]:
plt.bar(length_df.index,length_df['world_status <lambda>']['<lambda>'])
plt.title("Proportion perfect reconstruction")
plt.ylabel("Proportion perfect reconstruction")
plt.xlabel("Sequence length")
plt.show()

Ys = length_df['all_sequences_planning_cost sum']['mean']
CI95s = np.array([list(x) for x in length_df['all_sequences_planning_cost sum']['CI95']]).T
plt.bar(length_df.index,Ys,yerr=np.array([abs(Ys - CI95s[0]),abs(Ys - CI95s[1])]))
plt.title("Mean sum total planning cost over all sequences")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

Ys = length_df['all_sequences_planning_cost mean']['mean']
CI95s = np.array([list(x) for x in length_df['all_sequences_planning_cost mean']['CI95']]).T
plt.bar(length_df.index,Ys,yerr=np.array([abs(Ys - CI95s[0]),abs(Ys - CI95s[1])]))
plt.title("Mean mean total planning cost over all sequences")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

Ys = length_df['planning_cost sum']['mean']
CI95s = np.array([list(x) for x in length_df['planning_cost sum']['CI95']]).T
plt.bar(length_df.index,Ys,yerr=np.array([abs(Ys - CI95s[0]),abs(Ys - CI95s[1])]))
plt.title("Mean sum of planning costs for chosen sequence")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

Ys = length_df['planning_cost mean']['mean']
CI95s = np.array([list(x) for x in length_df['planning_cost mean']['CI95']]).T
plt.bar(length_df.index,Ys,yerr=np.array([abs(Ys - CI95s[0]),abs(Ys - CI95s[1])]))
plt.title("Mean mean of planning costs for chosen sequence")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

Ys = length_df['partial_solution_cost mean']['mean']
CI95s = np.array([list(x) for x in length_df['partial_solution_cost mean']['CI95']]).T
plt.bar(length_df.index,Ys,yerr=np.array([abs(Ys - CI95s[0]),abs(Ys - CI95s[1])]))
plt.title("Mean solution cost")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

Ys = length_df['decomposed_silhouette count']['mean']
CI95s = np.array([list(x) for x in length_df['decomposed_silhouette count']['CI95']]).T
plt.bar(length_df.index,Ys,yerr=np.array([abs(Ys - CI95s[0]),abs(Ys - CI95s[1])]))
plt.title("Mean number of subgoals")
plt.ylabel("Number of subgoals acted out")
plt.xlabel("Sequence length")
plt.show()


---

## $\lambda$

In [ ]:
cw_df = wfdf.groupby(['sequence_length item','c_weight item']).agg({
    'world_status <lambda>' : lambda x:len([r for r in x if r == 'Win'])/len(x),
     'partial_planning_cost sum':['mean',CI95],
     'partial_solution_cost sum':['mean',CI95],
     'partial_solution_cost mean':['mean',CI95],
     'planning_cost sum':['mean',CI95],
     'planning_cost mean':['mean',CI95],
#      'solution_cost':'mean', #includes penalty and therefore is meaningless
     'all_sequences_planning_cost sum':['mean',CI95],
         'all_sequences_planning_cost mean':['mean',CI95],
    'decomposed_silhouette count' : ['mean',CI95]
})

In [ ]:
cw_df

In [ ]:
for index in cw_df.index.get_level_values(0).unique():
    plt.plot(cw_df['world_status <lambda>']['<lambda>'][index],label=index)
    plt.title("Proportion perfect reconstruction")
    plt.ylabel("Proportion perfect reconstruction")
    plt.xlabel("$\lambda$")
    plt.legend()
plt.show()

for index in cw_df.index.get_level_values(0).unique():
    column = 'all_sequences_planning_cost sum'
    CIs = np.array([list(x) for x in cw_df[column]['CI95'][index]]).T
    Xs = cw_df[column]['mean'][index].index
    Ys = cw_df[column]['mean'][index]
    Error = np.array([Ys - CIs[0],Ys + CIs[1]])
    plt.errorbar(Xs,
                 Ys,
#                  yerr=Error,
                 label=index)
    plt.fill_between(Xs, CIs[0], CIs[1],alpha=0.3)
    plt.title("Mean sum total planning cost over all sequences")
    plt.ylabel("States evaluated")
    plt.xlabel("$\lambda$")
    plt.legend()
plt.show()

for index in cw_df.index.get_level_values(0).unique():
    column = 'all_sequences_planning_cost mean'
    CIs = np.array([list(x) for x in cw_df[column]['CI95'][index]]).T
    Xs = cw_df[column]['mean'][index].index
    Ys = cw_df[column]['mean'][index]
    Error = np.array([Ys - CIs[0],Ys + CIs[1]])
    plt.errorbar(Xs,
                 Ys,
#                  yerr=Error,
                 label=index)
    plt.fill_between(Xs, CIs[0], CIs[1],alpha=0.3)
    plt.title("Mean mean total planning cost over all sequences")
    plt.ylabel("States evaluated")
    plt.xlabel("$\lambda$")
    plt.legend()
plt.show()

for index in cw_df.index.get_level_values(0).unique():
    column = 'partial_planning_cost sum'
    CIs = np.array([list(x) for x in cw_df[column]['CI95'][index]]).T
    Xs = cw_df[column]['mean'][index].index
    Ys = cw_df[column]['mean'][index]
    Error = np.array([Ys - CIs[0],Ys + CIs[1]])
    plt.errorbar(Xs,
                 Ys,
#                  yerr=Error,
                 label=index)
    plt.fill_between(Xs, CIs[0], CIs[1],alpha=0.3)
    plt.title("Mean sum of partial planning costs for chosen sequence")
    plt.ylabel("States evaluated")
    plt.xlabel("$\lambda$")
    plt.legend()
plt.show()

for index in cw_df.index.get_level_values(0).unique():
    column = 'planning_cost sum'
    CIs = np.array([list(x) for x in cw_df[column]['CI95'][index]]).T
    Xs = cw_df[column]['mean'][index].index
    Ys = cw_df[column]['mean'][index]
    Error = np.array([Ys - CIs[0],Ys + CIs[1]])
    plt.errorbar(Xs,
                 Ys,
#                  yerr=Error,
                 label=index)
    plt.fill_between(Xs, CIs[0], CIs[1],alpha=0.3)
    plt.title("Mean sum of planning costs for chosen sequence")
    plt.ylabel("States evaluated")
    plt.xlabel("$\lambda$")
    plt.legend()
plt.show()

for index in cw_df.index.get_level_values(0).unique():
    column = 'partial_solution_cost mean'
    CIs = np.array([list(x) for x in cw_df[column]['CI95'][index]]).T
    Xs = cw_df[column]['mean'][index].index
    Ys = cw_df[column]['mean'][index]
    Error = np.array([Ys - CIs[0],Ys + CIs[1]])
    plt.errorbar(Xs,
                 Ys,
#                  yerr=Error,
                 label=index)
    plt.fill_between(Xs, CIs[0], CIs[1],alpha=0.3)
    plt.title("Mean solution cost")
    plt.ylabel("States evaluated")
    plt.xlabel("$\lambda$")
    plt.legend()
plt.show()

for index in cw_df.index.get_level_values(0).unique():
    column = 'decomposed_silhouette count'
    CIs = np.array([list(x) for x in cw_df[column]['CI95'][index]]).T
    Xs = cw_df[column]['mean'][index].index
    Ys = cw_df[column]['mean'][index]
    Error = np.array([Ys - CIs[0],Ys + CIs[1]])
    plt.errorbar(Xs,
                 Ys,
#                  yerr=Error,
                 label=index)
    plt.fill_between(Xs, CIs[0], CIs[1],alpha=0.3)
    plt.title("Mean number of subgoals")
    plt.ylabel("Number of subgoals acted out")
    plt.xlabel("$\lambda$")
    plt.legend()
plt.show()



---

## Sandgraph! 🏝

Since loading the pickled data takes too long, we sketchily recreate the names from the string of the decomposition

In [ ]:
import re
import ast
def str2array(s):
    #strip "array" and parentheses
    s=re.sub('\[array\(', '', s.strip())
    s=re.sub('\)]', '', s.strip())
    # Remove space after [
    s=re.sub('\[ +', '[', s.strip())
    # Replace commas and spaces
    s=re.sub('[,\s]+', ', ', s)
    return np.array(ast.literal_eval(s))

In [ ]:
from matplotlib.patches import Rectangle
from matplotlib import cm

In [ ]:
cmap = cm.viridis.colors
def get_color(index,max=8):
    return cmap[round(index/max * (len(cmap)-1))]

Let's plot

In [ ]:
#lambda first

lambda_sorted_oneadf = oneadf.sort_values(['c_weight','world']) #sorted by lambda and world
# lambda_sorted_oneadf = oneadf.sort_values(['world','c_weight']) #sorted by lambda and world

for las in lambda_sorted_oneadf['sequence_length'].unique():
    #get names
    names = []
    for run in lambda_sorted_oneadf.query('sequence_length == @las').groupby('run_ID',sort=False):
        silhouettes = run[1]['decomposed_silhouette'].dropna()
        _names = [np.sum(str2array(s).sum(axis=1) != 0) for s in silhouettes]
        names.append(_names)
#         print(run[1]['c_weight'].head(1).item(),run[1]['world'].head(1).item())
    # plot the sand graph
    plt.plot()
    for x,n in enumerate(names):
        for i,g in enumerate(reversed(n)):
            plt.gca().add_patch(Rectangle((x,0),1,g, facecolor = get_color(len(n) - i)))
    plt.title("Lookahead: "+str(round(las-1,0)))
    plt.xlabel("Iteration sorted by $\lambda$")
    # add back lambda ticks
    lambdas = list(lambda_sorted_oneadf['c_weight'].unique())
    plt.xticks(np.linspace(0,len(names),len(lambdas) + 1), lambdas+[""]) #assuming we have equally many observations for each lambda
    plt.ylabel("Vertical location of subgoal")
    plt.show()

In [ ]:
#world first

lambda_sorted_oneadf = oneadf.sort_values(['world','c_weight']) #sorted by lambda and world

for las in lambda_sorted_oneadf['sequence_length'].unique():
    #get names
    names = []
    worlds = []
    for run in lambda_sorted_oneadf.query('sequence_length == @las').groupby('run_ID',sort=False):
        silhouettes = run[1]['decomposed_silhouette'].dropna()
        _names = [np.sum(str2array(s).sum(axis=1) != 0) for s in silhouettes]
        names.append(_names)
        worlds.append(run[1]['world'].head(1).item())
#         print(run[1]['c_weight'].head(1).item(),run[1]['world'].head(1).item())
    # plot the sand graph
    plt.plot()
    last_world = worlds[0]
    for x,n in enumerate(names):
        for i,g in enumerate(reversed(n)):
            plt.gca().add_patch(Rectangle((x,0),1,g, facecolor = get_color(len(n) - i)))
        # draw world demarkator
        if worlds[x] != last_world:         
                        plt.gca().add_patch(Rectangle((x,-.2),len(names)/256,8.4, facecolor = "grey"))
        last_world = worlds[x]
    plt.title("Lookahead: "+str(round(las-1,0)))
#     plt.xlabel("Iteration sorted by $\lambda$")
    # add back lambda ticks
    lambdas = list(lambda_sorted_oneadf['c_weight'].unique())
#     plt.xticks(np.linspace(0,len(names),len(lambdas) + 1), lambdas+[""]) #assuming we have equally many observations for each lambda
    plt.ylabel("Vertical location of subgoal")
    plt.show()